In [ ]:
url = "https://server.workshop.vantage6.ai"
port = 443
api_path = "/api"

# When running this, create a file config.py with the following content:
from config import username, password

In [ ]:
from vantage6.client import Client

client = Client(url, port, api_path, log_level='debug')

client.authenticate(username, password)
client.setup_encryption(None)

In [ ]:
from workshop_resources.organizations import organizations as organization_names
from workshop_resources.collaborations import collaborations as collaboration_names

In [ ]:
### read participants as list of dictionaries
import csv

participants_file = 'workshop_resources/participants.csv'
participants = []
with open(participants_file, mode='r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        participants.append(row)


In [ ]:
# clear users
for participant in participants:
    response = client.user.list(firstname=participant['first_name'], lastname=participant['last_name'])
    if len(response['data']) > 0:
        user = response['data'][0]
        client.user.delete(user['id'])

In [ ]:
# clear organizations
for organization in organization_names:
    response = client.organization.list(name=organization['name'])
    if len(response['data']) > 0:
        org = response['data'][0]
        client.organization.delete(org['id'], delete_dependents=True)

In [ ]:
# note that this also deletes the studies and the nodes
# TODO there is a bug. Therefore studies are not deleted by this endpoint. See
#      https://github.com/vantage6/vantage6/issues/1465
for collab in client.collaboration.list(scope="global")['data']:
    if collab['name'] in collaboration_names:
        client.collaboration.delete(collab['id'], delete_dependents=True)

for study in client.study.list()["data"]:
    client.study.delete(study["id"])

# Delete dummy resources

In [ ]:
for participant in participants:

    abbrev_name = participant['first_name'][0].lower() + participant['last_name'].lower()

    try:
        user = client.user.list(username=f"{abbrev_name}_admin")
        client.user.delete(user['data'][0]['id'])
    except Exception as e:
        print("Failed to delete user", e)

    try:
        org1 = client.organization.list(name=f"{abbrev_name}_main_org")
        org2 = client.organization.list(name=f"{abbrev_name}_sub_org1")
        org3 = client.organization.list(name=f"{abbrev_name}_sub_org2")

        client.organization.delete(org1['data'][0]['id'], delete_dependents=True)
        client.organization.delete(org2['data'][0]['id'], delete_dependents=True)
        client.organization.delete(org3['data'][0]['id'], delete_dependents=True)

    except Exception as e:
        print("Failed to delete organization", e)

    try:
        collab1 = client.collaboration.list(scope="global", name=f"{abbrev_name}_collab1")
        collab2 = client.collaboration.list(scope="global", name=f"{abbrev_name}_collab2")
        client.collaboration.delete(collab1['data'][0]['id'], delete_dependents=True)
        client.collaboration.delete(collab2['data'][0]['id'], delete_dependents=True)
    except Exception as e:
        print("Failed to delete collaboration", e)
